# Collecting Reddit Data from r/WallStreetBets

## Part 1: Setup

### Install Packages

In [ ]:
# Install the packages if they don't exist.
%pip install praw
%pip install pyarrow
%pip install python-dotenv

# Import Packages
import praw, time, os, pyarrow
from IPython.display import display
from dotenv import load_dotenv, dotenv_values
from requests import Session
import pandas as pd
from IPython import get_ipython
from tqdm import tqdm 

# Load environment variables from .env file
load_dotenv('.env')
config = dotenv_values()

# Get config from colab or other environment.
def is_colab():
    return get_ipython().__class__.__module__ == "google.colab._shell"

if is_colab():
    from google.colab import userdata
    config = {}
    config['CLIENT_SECRET'] = userdata.get('CLIENT_SECRET')
    config['CLIENT_ID'] = userdata.get('CLIENT_ID')
    config['NAME'] = userdata.get('NAME')
    config['REDIRECT_URI'] = userdata.get('REDIRECT_URI')
    config['USERNAME'] = userdata.get('USERNAME')
    config['PASSWORD'] = userdata.get('PASSWORD')

else:
    load_dotenv('.env')
    config = dotenv_values()


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Part 2: Collecting Submissions from Reddit

### Open Reddit Connection

In [2]:
# Create a custom session with a timeout
session = Session()
session.headers.update({'User-Agent': 'praw'})
session.timeout = 10  # Set a timeout of 10 seconds

# Login to Reddit using PRAW
reddit = praw.Reddit(
    client_id=config['CLIENT_ID'],
    client_secret=config['CLIENT_SECRET'],
    requestor_kwargs={"session": session},
    username=config['USERNAME'],
    password=config['PASSWORD'],
    user_agent="CS470 ML Project Access by u/GregorybLafayetteML"
)

# Add some peripheral config data
reddit.config.log_requests = 1
reddit.config.store_json_result = True

# Test the connection
try:
    username = reddit.user.me()
    print("Successfully logged in to Reddit!")
    print(f"Logged in as: u/{username}")
except Exception as e:
    print(f"Failed to log in: {e}")

Successfully logged in to Reddit!
Logged in as: u/GregorybLafayetteML


### Accessing Reddit Data

To access reddit posts, we'll need send a request with the number of post we want to get. The following example finds the top 10 hottest posts on the u/wallstreetbets subreddit. We'll show the post title, score, flair, and URL.

In [3]:
top_posts = reddit.subreddit('wallstreetbets').hot(limit=10)
print("Top 10 hot posts from r/wallstreetbets:")
for post in top_posts:
    print(f"Title: {post.title}, Score: {post.score}, Flair: {post.link_flair_text}, URL: {post.url}")

Top 10 hot posts from r/wallstreetbets:
Title: Weekly Earnings Thread 4/21 - 4/25, Score: 269, Flair: Earnings Thread, URL: https://i.redd.it/r0jnwrc7neve1.jpeg
Title: What Are Your Moves Tomorrow, April 24, 2025, Score: 186, Flair: Daily Discussion, URL: https://www.reddit.com/r/wallstreetbets/comments/1k68qbf/what_are_your_moves_tomorrow_april_24_2025/
Title: TESLA is forming a nice descending triangle. Your time is coming tesla bears!, Score: 2095, Flair: Discussion, URL: https://i.redd.it/jy8wfil83mwe1.png
Title: $500 to $18k overnight (3,000% gain), Score: 2655, Flair: Gain, URL: https://www.reddit.com/gallery/1k60455
Title: Retailers I work with are already projecting 30%+ revenue loss over 2025. We haven't even begun to feel the damage from tariffs yet., Score: 4996, Flair: Discussion, URL: https://www.reddit.com/r/wallstreetbets/comments/1k5vy0i/retailers_i_work_with_are_already_projecting_30/
Title: US economic output hits 16-month low in April, expectations crater, Score: 104

For this project, we'll need far more than ten posts at a time. The reddit API will limit our access to 100 posts at a time. Fortunately, the api uses a ListingGenerator which allows us to access our metered connection in sequential blocks. The following example shows how we can utilize this behavior, grabbing blocks of 100 posts at a time. In our example, we'll grab blocks of posts until we reach 5000 posts or our access times out. Notice that the procedure ends early with around 750-800 posts collected. The results are sparce, because our connection either timed out or was metered down by reddit. The latter option is more likely.

In [4]:
# Access the subreddit
subreddit = reddit.subreddit("wallstreetbets")

# Initialize variables
batch_size = 50 # Number of posts per batch
total_posts = 5000  # Total number of posts to fetch
all_posts = []  # To store all the retrieved posts
after = None  # To keep track of the last post for pagination

# Fetch posts in batches
while len(all_posts) < total_posts:
    # Fetch the next batch of posts
    submissions = subreddit.new(limit=batch_size, params={"after": after})

    batch_posts = []
    for submission in submissions:
        batch_posts.append(submission)

        # Update the `after` variable with the last submission's fullname
        after = submission.fullname

    # Add the batch to the main list
    all_posts.extend(batch_posts)

    # Exit loop if no more posts are available
    if not batch_posts:
        print("No more posts to fetch.")
        break

    # Optional delay to avoid rate limits
    time.sleep(5)  # Adjust the delay as necessary

# Process the data (example: print the total number of posts fetched)
print(f"Fetched {len(all_posts)} posts in total.")

No more posts to fetch.
Fetched 791 posts in total.


Now that we have collected a large portion of posts/submssions, we'll parse the results and construct a dataframe with this data. We're going to collect more fields from this data than we might need right now, avoiding data limitations in the future.

In [5]:
# Parse are submission objects that we collected.
fields = ('title',
          'created_utc',
          'id',
          'is_original_content',
          'link_flair_text',
          'locked',
          'name',
          'num_comments',
          'over_18',
          'permalink',
          'selftext',
          'spoiler',
          'upvote_ratio')
list_of_submissions = []

# Parse each submission into a dictionary of the lised fields.
for submission in all_posts:
    full = vars(submission)
    sub_dict = {field:full[field] for field in fields}
    list_of_submissions.append(sub_dict)

# Create a python dataframe of these submissions.
collected_data = pd.DataFrame.from_records(list_of_submissions)

# Display the dataframe.
display(collected_data)

,title,created_utc,id,is_original_content,link_flair_text,locked,name,num_comments,over_18,permalink,selftext,spoiler,upvote_ratio
0,"""Venetian Risk Analyst, Circa 1492""\nWatching ...",1.745457e+09,1k6fm5u,False,Meme,False,t3_1k6fm5u,1,False,/r/wallstreetbets/comments/1k6fm5u/venetian_ri...,https://preview.redd.it/7w8rff9cmowe1.png?widt...,False,1.00
1,Holy cow this casino is a trip,1.745456e+09,1k6fic5,False,Gain,False,t3_1k6fic5,6,False,/r/wallstreetbets/comments/1k6fic5/holy_cow_th...,,False,1.00
2,"Can someone explain, some valuation expert say...",1.745456e+09,1k6ffo0,False,Discussion,False,t3_1k6ffo0,13,False,/r/wallstreetbets/comments/1k6ffo0/can_someone...,?,False,0.80
3,"… she is a rebel, she will do whatever she want",1.745455e+09,1k6f8kj,False,Meme,False,t3_1k6f8kj,2,False,/r/wallstreetbets/comments/1k6f8kj/she_is_a_re...,,False,0.36
4,Anyone else enjoying this volatility?,1.745454e+09,1k6es30,False,YOLO,False,t3_1k6es30,39,False,/r/wallstreetbets/comments/1k6es30/anyone_else...,All this tariff talk over the last couple of m...,False,0.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...
786,YOLO LFG!!!! 🚀🚀🚀,1.743791e+09,1jrivte,False,YOLO,False,t3_1jrivte,82,False,/r/wallstreetbets/comments/1jrivte/yolo_lfg/,,False,0.89
787,Quick in and out TSLA put,1.743791e+09,1jrit3u,False,Gain,False,t3_1jrit3u,9,False,/r/wallstreetbets/comments/1jrit3u/quick_in_an...,,False,0.92
788,Quarter million+ gain on QQQ puts,1.743791e+09,1jriqim,False,Gain,False,t3_1jriqim,53,False,/r/wallstreetbets/comments/1jriqim/quarter_mil...,"On January 15, 2025 I purchased 113 QQQ puts f...",False,0.96
789,"It ain't much, but it's honest work. $200 -> $7k",1.743790e+09,1jrie4w,False,Gain,False,t3_1jrie4w,13,False,/r/wallstreetbets/comments/1jrie4w/it_aint_muc...,,False,0.98


### Saving Reddit Data

In [6]:
# Save the collected data to parquet format
SUBMISSION_PARQUET_PATH = './data/wallstreetbets-collection.parquet'

# Create a pyarrow schema for the data types.
submission_schema = pyarrow.schema([
    ('title', pyarrow.string()),
    ('created_utc', pyarrow.float64()),
    ('id', pyarrow.string()),
    ('is_original_content', pyarrow.bool_()),
    ('link_flair_text', pyarrow.string()),
    ('locked', pyarrow.bool_()),
    ('name', pyarrow.string()),
    ('num_comments', pyarrow.int64()),
    ('over_18', pyarrow.bool_()),
    ('permalink', pyarrow.string()),
    ('selftext', pyarrow.string()),
    ('spoiler', pyarrow.bool_()),
    ('upvote_ratio', pyarrow.float64()),
])

In [7]:
# If the parqet does not exist, create it.
if not os.path.exists(SUBMISSION_PARQUET_PATH):
    collected_data.to_parquet(SUBMISSION_PARQUET_PATH, engine='pyarrow', schema=submission_schema)

# If the data file already exist, merge new data with the existing one.
else:
    old_parquet = pd.read_parquet(SUBMISSION_PARQUET_PATH, engine='pyarrow', schema=submission_schema)
    new_parquet = pd.concat([old_parquet, collected_data])
    new_parquet = new_parquet.drop_duplicates(subset=['id','title','created_utc','name','permalink'], keep='last').reset_index(drop=True)
    new_parquet.to_parquet(SUBMISSION_PARQUET_PATH, engine='pyarrow', schema=submission_schema)

# Use the new collected data to get comment stuff.
SUBMISSION_PARQUET_PATH = './data/wallstreetbets-collection.parquet'
submission_collection = pd.read_parquet(SUBMISSION_PARQUET_PATH, engine='pyarrow', schema=submission_schema)
display(submission_collection)

,title,created_utc,id,is_original_content,link_flair_text,locked,name,num_comments,over_18,permalink,selftext,spoiler,upvote_ratio
0,Nivea Along,1.744832e+09,1k0t4jk,False,YOLO,False,t3_1k0t4jk,5,False,/r/wallstreetbets/comments/1k0t4jk/nivea_along/,After -7% yesterday and -10% today,False,0.67
1,Powell to Volatile Stock Market: You’re on You...,1.744836e+09,1k0unbq,False,News,False,t3_1k0unbq,2,False,/r/wallstreetbets/comments/1k0unbq/powell_to_v...,,False,0.86
2,Made back the last Wendy’s paycheck I lost,1.744834e+09,1k0tv2y,False,Gain,False,t3_1k0tv2y,6,False,/r/wallstreetbets/comments/1k0tv2y/made_back_t...,,False,0.94
3,After market observation. When I finished buyi...,1.744833e+09,1k0tnqx,False,Gain,False,t3_1k0tnqx,8,False,/r/wallstreetbets/comments/1k0tnqx/after_marke...,https://preview.redd.it/41ilvj6f39ve1.png?widt...,False,0.72
4,Ominous,1.744833e+09,1k0thnd,False,Discussion,False,t3_1k0thnd,110,False,/r/wallstreetbets/comments/1k0thnd/ominous/,NVIDIA 2024 is starting to rhyme like Cisco 20...,False,0.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1064,YOLO LFG!!!! 🚀🚀🚀,1.743791e+09,1jrivte,False,YOLO,False,t3_1jrivte,82,False,/r/wallstreetbets/comments/1jrivte/yolo_lfg/,,False,0.89
1065,Quick in and out TSLA put,1.743791e+09,1jrit3u,False,Gain,False,t3_1jrit3u,9,False,/r/wallstreetbets/comments/1jrit3u/quick_in_an...,,False,0.92
1066,Quarter million+ gain on QQQ puts,1.743791e+09,1jriqim,False,Gain,False,t3_1jriqim,53,False,/r/wallstreetbets/comments/1jriqim/quarter_mil...,"On January 15, 2025 I purchased 113 QQQ puts f...",False,0.96
1067,"It ain't much, but it's honest work. $200 -> $7k",1.743790e+09,1jrie4w,False,Gain,False,t3_1jrie4w,13,False,/r/wallstreetbets/comments/1jrie4w/it_aint_muc...,,False,0.98


## Part 3: Collecting Comments from Reddit

### Creating a database of reddit threads

In [8]:
# Use the same methofology whih we used to collect submissions, but we'll add a parent submission id. and parent comment id.
# Since the comment section can be very deep, we'll limit comments to a breadth of 10.
# This may still be a lot more comments than we need for larger discussions.
def extract_comments_from_submission(submission_id: str):
    try:
        submission = reddit.submission(id=submission_id)
        submission.comments.replace_more(limit=10)  # Limit to 10 levels of comments
        comments = []

        for comment in submission.comments.list():
            if isinstance(comment, praw.models.MoreComments):
                continue

            # NOTE: It looks like the top comment may be a user report. We'll ignore if is has certain text.
            SKIPTEXT = '**User Report**'
            if SKIPTEXT in comment.body:
                continue

            # Append the comment data to the list
            comments.append({
                'parent_post_id': submission_id,
                'parent_comment_id': comment.parent_id,
                'comment_id': comment.id,
                'author': str(comment.author),
                'created_utc': comment.created_utc,
                'score': comment.score,
                'body': comment.body
            })

        return comments

    except Exception as e:
        # Get the HTTP error code if available
        if hasattr(e, 'response') and e.response is not None:
            error_code = e.response.status_code
            print(f"HTTP Error {error_code} while fetching comments for submission {submission_id}")
        else:
            error_code = None

        # Print the an erroor message and return nothing.
        print(f"Error fetching comments for submission {submission_id}: {e}")
        return []



In [9]:
# Show the results from one submission's comments
submission_id = submission_collection.iloc[0]['id']

# How many actual comments are there for this submission?
submission = reddit.submission(id=submission_id)
print(f"Submission ID: {submission_id}")
print(f"Title: {submission.title}")
print(f"Number of comments: {submission.num_comments}")

# Get the comments for the submission
results = extract_comments_from_submission(submission_id)

# Create a dataframe of the comments
comments_df = pd.DataFrame(results)

# Display the comments dataframe
display(comments_df)

Submission ID: 1k0t4jk
Title: Nivea Along
Number of comments: 6


,parent_post_id,parent_comment_id,comment_id,author,created_utc,score,body
0,1k0t4jk,t3_1k0t4jk,mngmxdi,chrissurra,1.744832e+09,5,You mean Neveah
1,1k0t4jk,t3_1k0t4jk,mngn956,Alert_Barber_3105,1.744832e+09,5,The skin cream?
2,1k0t4jk,t3_1k0t4jk,mngoka2,Reasonable_Roger,1.744832e+09,1,Psoriasis calls
3,1k0t4jk,t1_mngmxdi,mngnbpk,Own-Foundation3873,1.744832e+09,1,yesss sir
4,1k0t4jk,t1_mngn956,mngnj1n,Own-Foundation3873,1.744832e+09,1,pow till it creams


In [10]:
# Collect the comments for all the submissions.
all_comments = []
for submission in submission_collection['id']:
    comments = extract_comments_from_submission(submission)
    all_comments.extend(comments)
    time.sleep(1)  # Optional delay to avoid rate limits

# Create a python dataframe of these comments.
comments_df = pd.DataFrame.from_records(all_comments)
display(comments_df)

,parent_post_id,parent_comment_id,comment_id,author,created_utc,score,body
0,1k0t4jk,t3_1k0t4jk,mngmxdi,chrissurra,1.744832e+09,4,You mean Neveah
1,1k0t4jk,t3_1k0t4jk,mngn956,Alert_Barber_3105,1.744832e+09,5,The skin cream?
2,1k0t4jk,t3_1k0t4jk,mngoka2,Reasonable_Roger,1.744832e+09,1,Psoriasis calls
3,1k0t4jk,t1_mngmxdi,mngnbpk,Own-Foundation3873,1.744832e+09,1,yesss sir
4,1k0t4jk,t1_mngn956,mngnj1n,Own-Foundation3873,1.744832e+09,1,pow till it creams
...,...,...,...,...,...,...,...
208072,1jrie4w,t1_mlh9p19,mlikdz9,UserHistoryIrelevent,1.743845e+09,1,Hey can i dm you? I am confused on how puts work
208073,1jrie4w,t1_mlglt70,mlgmqxf,nekromantiks,1.743811e+09,3,![img](emote|t5_2th52|27189)
208074,1jrie4w,t1_mlikdz9,mlk0hvn,nekromantiks,1.743868e+09,5,You don't want my advice I promise\n\nI think ...
208075,1jridsd,t3_1jridsd,mlewv67,CruxDelt4,1.743790e+09,1,What does Spy have to do with Nintendo?


In [11]:
# Save the collected data to parquet format
COMMENT_PARQUET_PATH = './data/wallstreetbets-comment-collection.parquet'

# Create a pyarrow schema for the comment data
comment_schema = pyarrow.schema([
    ('parent_post_id', pyarrow.string()),
    ('parent_comment_id', pyarrow.string()),
    ('comment_id', pyarrow.string()),
    ('author', pyarrow.string()),
    ('created_utc', pyarrow.float64()),
    ('score', pyarrow.int64()),
    ('body', pyarrow.string())
])

In [12]:
# Write the comments to parquet file. If it exists, append to it.
if not os.path.exists(COMMENT_PARQUET_PATH):
    comments_df.to_parquet(COMMENT_PARQUET_PATH, engine='pyarrow', schema=comment_schema)
else:
    old_parquet = pd.read_parquet(COMMENT_PARQUET_PATH, engine='pyarrow', schema=comment_schema)
    new_parquet = pd.concat([old_parquet, comments_df])
    new_parquet = new_parquet.drop_duplicates(subset=['parent_post_id','parent_comment_id','author','created_utc'], keep='last').reset_index(drop=True)
    new_parquet.to_parquet(COMMENT_PARQUET_PATH, engine='pyarrow', schema=comment_schema)